## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.


## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [1]:
import pandas as pd
import os

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns

###
import numpy as np
from sklearn.decomposition import PCA

In [3]:
# !pip install pandas --upgrade
# !pip install git+https://github.com/pandas-dev/pandas.git --user
# print(pd.__version__)

## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [4]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

df = pd.read_csv(os.path.join('data/train.csv'))

print("The shape of the dataset is {}.\n\n".format(df.shape))

df.head()

The shape of the dataset is (6407, 16).




,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,Severity,timestamp
0,0,37.762150,-122.405660,False,0.044,False,False,False,False,False,False,False,True,R,2,2016-03-25 15:13:02
1,1,37.719157,-122.448254,False,0.000,False,False,False,False,False,False,False,False,R,2,2020-05-05 19:23:00
2,2,37.808498,-122.366852,False,0.000,False,False,False,False,False,False,True,False,R,3,2016-09-16 19:57:16
3,3,37.785930,-122.391080,False,0.009,False,False,True,False,False,False,False,False,R,1,2020-03-29 19:48:43
4,4,37.719141,-122.448457,False,0.000,False,False,False,False,False,False,False,False,R,2,2019-10-09 08:47:00


We've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [5]:
df.drop(columns='ID').describe()

,Lat,Lng,Distance(mi),Severity
count,6407.000000,6407.000000,6407.000000,6407.000000
mean,37.765653,-122.405990,0.135189,2.293429
std,0.032555,0.028275,0.396360,0.521225
min,37.609619,-122.510440,0.000000,1.000000
25%,37.737096,-122.412210,0.000000,2.000000
50%,37.768238,-122.404835,0.000000,2.000000
75%,37.787813,-122.392477,0.041000,3.000000
max,37.825626,-122.349734,6.820000,4.000000


In [6]:
df.isna().sum()

ID              0
Lat             0
Lng             0
Bump            0
Distance(mi)    0
Crossing        0
Give_Way        0
Junction        0
No_Exit         0
Railway         0
Roundabout      0
Stop            0
Amenity         0
Side            0
Severity        0
timestamp       0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6407 entries, 0 to 6406
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            6407 non-null   int64  
 1   Lat           6407 non-null   float64
 2   Lng           6407 non-null   float64
 3   Bump          6407 non-null   bool   
 4   Distance(mi)  6407 non-null   float64
 5   Crossing      6407 non-null   bool   
 6   Give_Way      6407 non-null   bool   
 7   Junction      6407 non-null   bool   
 8   No_Exit       6407 non-null   bool   
 9   Railway       6407 non-null   bool   
 10  Roundabout    6407 non-null   bool   
 11  Stop          6407 non-null   bool   
 12  Amenity       6407 non-null   bool   
 13  Side          6407 non-null   object 
 14  Severity      6407 non-null   int64  
 15  timestamp     6407 non-null   object 
dtypes: bool(9), float64(3), int64(2), object(2)
memory usage: 406.8+ KB


In [8]:
# count the values of each column
for col in df.columns:
    print(f'For column {col}')
    print('------------------')
    print(df[col].value_counts())
    print('\n')

For column ID
------------------
0       1
2668    1
645     1
2692    1
4735    1
       ..
5424    1
3371    1
1322    1
5416    1
2047    1
Name: ID, Length: 6407, dtype: int64


For column Lat
------------------
37.808498    265
37.752502    106
37.808110     94
37.808253     89
37.807710     74
            ... 
37.768990      1
37.725360      1
37.800937      1
37.729619      1
37.807962      1
Name: Lat, Length: 2061, dtype: int64


For column Lng
------------------
-122.366852    269
-122.367190     94
-122.366974     92
-122.367640     74
-122.403008     63
              ... 
-122.419891      1
-122.405582      1
-122.366943      1
-122.407829      1
-122.397790      1
Name: Lng, Length: 1937, dtype: int64


For column Bump
------------------
False    6407
Name: Bump, dtype: int64


For column Distance(mi)
------------------
0.000    3923
0.010     499
0.037     158
0.420      40
0.208      35
         ... 
0.886       1
0.186       1
0.386       1
1.720       1
0.235       1
N

In [9]:
df.drop(columns=["ID", "Bump", "Roundabout"], inplace=True)

In [10]:
corr_matrix = df.corr()
corr_matrix['Severity'].sort_values(ascending=False)

Severity        1.000000
Stop            0.229269
Lng             0.145313
Lat             0.099581
No_Exit        -0.007034
Give_Way       -0.012186
Distance(mi)   -0.013141
Railway        -0.033322
Junction       -0.068328
Amenity        -0.078915
Crossing       -0.090314
Name: Severity, dtype: float64

In [11]:
df.iloc[:, 3:10] = df.iloc[:, 3:10].astype(int)
df['Side'].replace('R', 0,inplace=True)
df['Side'].replace("L", 1,inplace=True)
df

,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp
0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,0,2,2016-03-25 15:13:02
1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,0,2,2020-05-05 19:23:00
2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,0,3,2016-09-16 19:57:16
3,37.785930,-122.391080,0.009,0,0,1,0,0,0,0,0,1,2020-03-29 19:48:43
4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,0,2,2019-10-09 08:47:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,37.740630,-122.407930,0.368,0,0,0,0,0,0,0,0,3,2017-10-01 18:36:13
6403,37.752755,-122.402790,0.639,0,0,1,0,0,0,0,0,2,2018-10-23 07:40:27
6404,37.726304,-122.446015,0.000,0,0,1,0,0,0,0,0,2,2019-10-28 15:45:00
6405,37.808090,-122.367211,0.000,0,0,1,0,0,0,0,0,3,2019-05-04 13:45:31


In [12]:
date_col = pd.to_datetime(df['timestamp']).dt

df["Year"] = date_col.year
df["Month"] = date_col.month
df["Day"] = date_col.day
df["Hour"] =  date_col.hour

hour = pd.to_datetime(df.timestamp).dt.hour
minute = pd.to_datetime(df.timestamp).dt.minute
sec = pd.to_datetime(df.timestamp).dt.second

df['Time(h)'] = hour + minute/60 + sec /(60*60)
df

,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp,Year,Month,Day,Hour,Time(h)
0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,0,2,2016-03-25 15:13:02,2016,3,25,15,15.217222
1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,0,2,2020-05-05 19:23:00,2020,5,5,19,19.383333
2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,0,3,2016-09-16 19:57:16,2016,9,16,19,19.954444
3,37.785930,-122.391080,0.009,0,0,1,0,0,0,0,0,1,2020-03-29 19:48:43,2020,3,29,19,19.811944
4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,0,2,2019-10-09 08:47:00,2019,10,9,8,8.783333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,37.740630,-122.407930,0.368,0,0,0,0,0,0,0,0,3,2017-10-01 18:36:13,2017,10,1,18,18.603611
6403,37.752755,-122.402790,0.639,0,0,1,0,0,0,0,0,2,2018-10-23 07:40:27,2018,10,23,7,7.674167
6404,37.726304,-122.446015,0.000,0,0,1,0,0,0,0,0,2,2019-10-28 15:45:00,2019,10,28,15,15.750000
6405,37.808090,-122.367211,0.000,0,0,1,0,0,0,0,0,3,2019-05-04 13:45:31,2019,5,4,13,13.758611


In [13]:
import xml.etree.ElementTree as ET

xml_data = open(os.path.join('data/holidays.xml'), 'r').read()
root = ET.XML(xml_data) 

data = []
cols = ["date", "description"]
for i, child in enumerate(root):
    data.append([subchild.text for subchild in child])

holidays_df = pd.DataFrame(data)  
holidays_df.columns = cols
holidays_df
# holidays_df = pd.read_xml(os.path.join(dataset_path, 'holidays.xml'))
# holidays_df.head()

,date,description
0,2012-01-02,New Year Day
1,2012-01-16,Martin Luther King Jr. Day
2,2012-02-20,Presidents Day (Washingtons Birthday)
3,2012-05-28,Memorial Day
4,2012-07-04,Independence Day
...,...,...
85,2020-09-07,Labor Day
86,2020-10-12,Columbus Day
87,2020-11-11,Veterans Day
88,2020-11-26,Thanksgiving Day


In [14]:
date_col = pd.to_datetime(holidays_df['date']).dt
holidays_df["Year"] = date_col.year
holidays_df["Month"] = date_col.month
holidays_df["Day"] = date_col.day
holidays_df.drop(columns=["date", "description"], inplace=True)

holidays_df['holiday']=1


holidays_df

,Year,Month,Day,holiday
0,2012,1,2,1
1,2012,1,16,1
2,2012,2,20,1
3,2012,5,28,1
4,2012,7,4,1
...,...,...,...,...
85,2020,9,7,1
86,2020,10,12,1
87,2020,11,11,1
88,2020,11,26,1


In [15]:
holidays_df.duplicated().sum()

0

In [16]:
df_plus_holidays = pd.merge(df, holidays_df, 
                    on=["Year","Month","Day"], how='left')
df_plus_holidays["holiday"].fillna(0, inplace=True)    
df_plus_holidays

,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,Severity,timestamp,Year,Month,Day,Hour,Time(h),holiday
0,37.762150,-122.405660,0.044,0,0,0,0,0,0,1,0,2,2016-03-25 15:13:02,2016,3,25,15,15.217222,0.0
1,37.719157,-122.448254,0.000,0,0,0,0,0,0,0,0,2,2020-05-05 19:23:00,2020,5,5,19,19.383333,0.0
2,37.808498,-122.366852,0.000,0,0,0,0,0,1,0,0,3,2016-09-16 19:57:16,2016,9,16,19,19.954444,0.0
3,37.785930,-122.391080,0.009,0,0,1,0,0,0,0,0,1,2020-03-29 19:48:43,2020,3,29,19,19.811944,0.0
4,37.719141,-122.448457,0.000,0,0,0,0,0,0,0,0,2,2019-10-09 08:47:00,2019,10,9,8,8.783333,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6402,37.740630,-122.407930,0.368,0,0,0,0,0,0,0,0,3,2017-10-01 18:36:13,2017,10,1,18,18.603611,0.0
6403,37.752755,-122.402790,0.639,0,0,1,0,0,0,0,0,2,2018-10-23 07:40:27,2018,10,23,7,7.674167,0.0
6404,37.726304,-122.446015,0.000,0,0,1,0,0,0,0,0,2,2019-10-28 15:45:00,2019,10,28,15,15.750000,0.0
6405,37.808090,-122.367211,0.000,0,0,1,0,0,0,0,0,3,2019-05-04 13:45:31,2019,5,4,13,13.758611,0.0


In [17]:
df_plus_holidays["holiday"].value_counts()

0.0    6259
1.0     148
Name: holiday, dtype: int64

In [18]:
weather_data = pd.read_csv(os.path.join('data/weather-sfcsv.csv'))
print("The shape of the dataset is {}.\n\n".format(weather_data.shape))
weather_data.head()

The shape of the dataset is (6901, 12).




,Year,Day,Month,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
0,2020,27,7,18,Fair,64.0,0.0,64.0,70.0,20.0,10.0,No
1,2017,30,9,17,Partly Cloudy,NaN,NaN,71.1,57.0,9.2,10.0,No
2,2017,27,6,5,Overcast,NaN,NaN,57.9,87.0,15.0,9.0,No
3,2016,7,9,9,Clear,NaN,NaN,66.9,73.0,4.6,10.0,No
4,2019,19,10,2,Fair,52.0,0.0,52.0,89.0,0.0,9.0,No


In [19]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6901 entries, 0 to 6900
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               6901 non-null   int64  
 1   Day                6901 non-null   int64  
 2   Month              6901 non-null   int64  
 3   Hour               6901 non-null   int64  
 4   Weather_Condition  6900 non-null   object 
 5   Wind_Chill(F)      3292 non-null   float64
 6   Precipitation(in)  3574 non-null   float64
 7   Temperature(F)     6899 non-null   float64
 8   Humidity(%)        6899 non-null   float64
 9   Wind_Speed(mph)    6556 non-null   float64
 10  Visibility(mi)     6900 non-null   float64
 11  Selected           6901 non-null   object 
dtypes: float64(6), int64(4), object(2)
memory usage: 647.1+ KB


In [20]:
print("duplicates =" ,weather_data.duplicated().sum())
weather_data.isnull().sum()

duplicates = 0


Year                    0
Day                     0
Month                   0
Hour                    0
Weather_Condition       1
Wind_Chill(F)        3609
Precipitation(in)    3327
Temperature(F)          2
Humidity(%)             2
Wind_Speed(mph)       345
Visibility(mi)          1
Selected                0
dtype: int64

In [21]:
weather_data = weather_data.dropna(subset=['Weather_Condition'])
weather_data.isnull().sum()

Year                    0
Day                     0
Month                   0
Hour                    0
Weather_Condition       0
Wind_Chill(F)        3608
Precipitation(in)    3326
Temperature(F)          1
Humidity(%)             1
Wind_Speed(mph)       344
Visibility(mi)          0
Selected                0
dtype: int64

In [22]:
# weather_data.head(60)
# for col in weather_data.columns:
#     print(f'For column {col}')
#     print('------------------')
#     print(weather_data[col].value_counts())
#     print('\n')

In [23]:
weather_data['Wind_Chill(F)'].fillna(int(weather_data['Wind_Chill(F)'].mean()), inplace=True)
weather_data['Temperature(F)'].fillna(weather_data['Temperature(F)'].median(), inplace=True)
weather_data['Humidity(%)'].fillna(int(weather_data['Humidity(%)'].median()), inplace=True)
weather_data['Wind_Speed(mph)'].fillna(weather_data['Wind_Speed(mph)'].mean(), inplace=True)
weather_data['Visibility(mi)'].fillna(weather_data['Visibility(mi)'].median(), inplace=True)


weather_data.head()

,Year,Day,Month,Hour,Weather_Condition,Wind_Chill(F),Precipitation(in),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Selected
0,2020,27,7,18,Fair,64.0,0.0,64.0,70.0,20.0,10.0,No
1,2017,30,9,17,Partly Cloudy,59.0,NaN,71.1,57.0,9.2,10.0,No
2,2017,27,6,5,Overcast,59.0,NaN,57.9,87.0,15.0,9.0,No
3,2016,7,9,9,Clear,59.0,NaN,66.9,73.0,4.6,10.0,No
4,2019,19,10,2,Fair,52.0,0.0,52.0,89.0,0.0,9.0,No


In [24]:
weather_data = weather_data.drop(columns=["Precipitation(in)","Selected"])
weather_data.shape

(6900, 10)

In [25]:
weather_data[["Year","Month", "Day", "Hour"]].duplicated().sum()

929

In [26]:
# grouped_df = weather_data.groupby(['Year', 'Month', 'Day', 'Hour'])

# for key,item in grouped_df:
    
#     a_group = grouped_df.get_group(key)
    
#     if a_group["Year"].count() > 1:
#         print(a_group, "\n")

In [27]:
from sklearn.preprocessing import OneHotEncoder


one_hot = OneHotEncoder()
encoded = one_hot.fit_transform(weather_data[['Weather_Condition']])


weather_data[one_hot.categories_[0]] = encoded.toarray()
weather_data

,Year,Day,Month,Hour,Weather_Condition,Wind_Chill(F),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),...,Overcast,Partly Cloudy,Partly Cloudy / Windy,Patches of Fog,Rain,Rain / Windy,Scattered Clouds,Shallow Fog,Smoke,Squalls
0,2020,27,7,18,Fair,64.0,64.0,70.0,20.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017,30,9,17,Partly Cloudy,59.0,71.1,57.0,9.2,10.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017,27,6,5,Overcast,59.0,57.9,87.0,15.0,9.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016,7,9,9,Clear,59.0,66.9,73.0,4.6,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019,19,10,2,Fair,52.0,52.0,89.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6896,2018,23,1,21,Clear,59.0,51.1,80.0,3.5,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6897,2019,16,6,7,Cloudy,56.0,56.0,80.0,9.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6898,2017,7,2,4,Rain,59.0,61.0,90.0,32.2,7.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6899,2016,22,4,16,Mostly Cloudy,59.0,61.0,67.0,21.9,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
weather_data.drop(columns=["Weather_Condition"], inplace=True)
weather_data = weather_data.groupby(['Year', 'Month', 'Day', 'Hour']).mean().reset_index()

weather_data.iloc[:, 10:36] = weather_data.iloc[:, 10:36].astype(bool).astype(int)
weather_data

,Year,Month,Day,Hour,Wind_Chill(F),Temperature(F),Humidity(%),Wind_Speed(mph),Visibility(mi),Clear,...,Overcast,Partly Cloudy,Partly Cloudy / Windy,Patches of Fog,Rain,Rain / Windy,Scattered Clouds,Shallow Fog,Smoke,Squalls
0,2016,3,23,9,59.0,55.45,69.5,10.813164,10.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2016,3,23,16,59.0,61.00,62.0,20.700000,10.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2016,3,23,20,59.0,59.00,67.0,12.700000,10.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,2016,3,24,8,59.0,55.00,80.0,6.900000,10.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,2016,3,24,9,59.0,55.90,77.0,3.500000,10.0,0.0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5966,2020,12,31,12,58.5,58.50,54.5,12.000000,10.0,0.0,...,0,1,0,0,0,0,0,0,0,0
5967,2020,12,31,14,60.5,60.50,52.5,12.000000,10.0,0.0,...,0,1,0,0,0,0,0,0,0,0
5968,2020,12,31,15,59.0,59.00,62.0,8.000000,10.0,0.0,...,0,0,0,0,0,0,0,0,0,0
5969,2020,12,31,18,54.0,54.00,77.0,7.000000,10.0,0.0,...,0,1,0,0,0,0,0,0,0,0


#PCA


In [29]:
###########
#performing pca on the weather dataset
pca = PCA(n_components=4)
pca.fit(weather_data)

columns = ['pca_%i' % i for i in range(4)]
df_pca = pd.DataFrame(pca.transform(weather_data), columns=columns, index=weather_data.index)
df_pca.head()

,pca_0,pca_1,pca_2,pca_3
0,-2.636996,7.504794,3.148185,-0.639108
1,7.607928,7.277221,-1.864652,9.067705
2,2.073336,7.301052,-0.571034,6.813062
3,-12.776269,8.482532,0.652729,-4.004339
4,-9.958385,8.503488,2.314813,-5.722873


In [30]:
extracted_col = weather_data[["Year","Month","Day", "Hour"]]
df_pca=df_pca.join(extracted_col)
df_pca.head()

,pca_0,pca_1,pca_2,pca_3,Year,Month,Day,Hour
0,-2.636996,7.504794,3.148185,-0.639108,2016,3,23,9
1,7.607928,7.277221,-1.864652,9.067705,2016,3,23,16
2,2.073336,7.301052,-0.571034,6.813062,2016,3,23,20
3,-12.776269,8.482532,0.652729,-4.004339,2016,3,24,8
4,-9.958385,8.503488,2.314813,-5.722873,2016,3,24,9


In [31]:
mergedDf = pd.merge(df_plus_holidays, df_pca, 
                    on=["Year","Month","Day", "Hour"], how='left')

mergedDf.shape

(6407, 23)

In [32]:
mergedDf.isnull().sum()

Lat             0
Lng             0
Distance(mi)    0
Crossing        0
Give_Way        0
Junction        0
No_Exit         0
Railway         0
Stop            0
Amenity         0
Side            0
Severity        0
timestamp       0
Year            0
Month           0
Day             0
Hour            0
Time(h)         0
holiday         0
pca_0           1
pca_1           1
pca_2           1
pca_3           1
dtype: int64

In [33]:
mergedDf.drop(columns=['Hour', 'timestamp'], inplace=True)
mergedDf = mergedDf.dropna()

In [34]:
corr_matrix = mergedDf.corr()
corr_matrix['Severity'].sort_values(ascending=False)

Severity        1.000000
Stop            0.229373
Lng             0.145331
Lat             0.099815
pca_3           0.010643
holiday         0.003168
Day            -0.000431
pca_1          -0.001141
No_Exit        -0.007033
Give_Way       -0.012183
Distance(mi)   -0.013071
Time(h)        -0.017812
Railway        -0.033292
Month          -0.049487
pca_2          -0.052138
Side           -0.060505
Junction       -0.068724
pca_0          -0.069466
Amenity        -0.078885
Crossing       -0.090264
Year           -0.171301
Name: Severity, dtype: float64

The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [35]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(mergedDf, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=['Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=['Severity'])
y_val = val_df['Severity']


As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

In [36]:
# # This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
# X_train = X_train[['Lat', 'Lng', 'Distance(mi)']]
# X_val = X_val[['Lat', 'Lng', 'Distance(mi)']]

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [37]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

Now let's test our classifier on the validation dataset and see the accuracy.

In [38]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

The accuracy of the classifier on the validation set is  0.750390015600624


Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [39]:
test_df = pd.read_csv(os.path.join('data/test.csv'))
test_df.head()

,ID,Lat,Lng,Bump,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,False,0.039,False,False,True,False,False,False,False,False,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,False,0.202,False,False,False,False,False,False,False,False,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,False,0.000,False,False,False,False,False,False,False,False,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,False,0.000,False,False,True,False,False,False,False,False,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,False,0.670,False,False,False,False,False,False,False,False,R,2018-10-17 09:54:58


In [40]:
test_df.shape

(1601, 15)

In [41]:
# # count the values of each column
# for col in test_df.columns:
#     print(f'For column {col}')
#     print('------------------')
#     print(test_df[col].value_counts())
#     print('\n')

In [42]:
test_df.drop(columns=["Bump", "Roundabout"], inplace=True)

test_df.iloc[:, 4:11] = test_df.iloc[:, 4:11].astype(int)
test_df

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,0,R,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,0,R,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,0,R,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,0,R,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,0,R,2018-10-17 09:54:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,8003,37.812973,-122.362335,4.460,0,0,0,0,0,0,0,R,2020-06-26 22:32:22
1597,8004,37.761818,-122.405861,0.010,0,0,1,0,0,0,0,R,2016-12-03 07:16:30
1598,8005,37.732260,-122.431970,0.431,0,0,1,0,0,0,0,R,2017-02-20 06:32:44
1599,8006,37.786782,-122.390126,0.000,1,0,0,0,0,0,0,R,2019-10-31 20:35:00


In [43]:
test_df['Side'].replace('R', 0,inplace=True)
test_df['Side'].replace("L", 1,inplace=True)
test_df

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,timestamp
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,0,0,2016-04-04 19:20:31
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,0,0,2020-10-28 11:51:00
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,0,0,2019-09-09 07:36:45
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,0,0,2019-08-06 15:46:25
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,0,0,2018-10-17 09:54:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,8003,37.812973,-122.362335,4.460,0,0,0,0,0,0,0,0,2020-06-26 22:32:22
1597,8004,37.761818,-122.405861,0.010,0,0,1,0,0,0,0,0,2016-12-03 07:16:30
1598,8005,37.732260,-122.431970,0.431,0,0,1,0,0,0,0,0,2017-02-20 06:32:44
1599,8006,37.786782,-122.390126,0.000,1,0,0,0,0,0,0,0,2019-10-31 20:35:00


In [44]:
date_col = pd.to_datetime(test_df['timestamp']).dt

test_df["Year"] = date_col.year
test_df["Month"] = date_col.month
test_df["Day"] = date_col.day
test_df["Hour"] =  date_col.hour
test_df

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,timestamp,Year,Month,Day,Hour
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,0,0,2016-04-04 19:20:31,2016,4,4,19
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,0,0,2020-10-28 11:51:00,2020,10,28,11
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,0,0,2019-09-09 07:36:45,2019,9,9,7
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,0,0,2019-08-06 15:46:25,2019,8,6,15
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,0,0,2018-10-17 09:54:58,2018,10,17,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,8003,37.812973,-122.362335,4.460,0,0,0,0,0,0,0,0,2020-06-26 22:32:22,2020,6,26,22
1597,8004,37.761818,-122.405861,0.010,0,0,1,0,0,0,0,0,2016-12-03 07:16:30,2016,12,3,7
1598,8005,37.732260,-122.431970,0.431,0,0,1,0,0,0,0,0,2017-02-20 06:32:44,2017,2,20,6
1599,8006,37.786782,-122.390126,0.000,1,0,0,0,0,0,0,0,2019-10-31 20:35:00,2019,10,31,20


In [45]:
hour = pd.to_datetime(test_df.timestamp).dt.hour
minute = pd.to_datetime(test_df.timestamp).dt.minute
sec = pd.to_datetime(test_df.timestamp).dt.second

test_df['Time(h)'] = hour + minute/60 + sec /(60*60)
test_df

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,timestamp,Year,Month,Day,Hour,Time(h)
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,0,0,2016-04-04 19:20:31,2016,4,4,19,19.341944
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,0,0,2020-10-28 11:51:00,2020,10,28,11,11.850000
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,0,0,2019-09-09 07:36:45,2019,9,9,7,7.612500
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,0,0,2019-08-06 15:46:25,2019,8,6,15,15.773611
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,0,0,2018-10-17 09:54:58,2018,10,17,9,9.916111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,8003,37.812973,-122.362335,4.460,0,0,0,0,0,0,0,0,2020-06-26 22:32:22,2020,6,26,22,22.539444
1597,8004,37.761818,-122.405861,0.010,0,0,1,0,0,0,0,0,2016-12-03 07:16:30,2016,12,3,7,7.275000
1598,8005,37.732260,-122.431970,0.431,0,0,1,0,0,0,0,0,2017-02-20 06:32:44,2017,2,20,6,6.545556
1599,8006,37.786782,-122.390126,0.000,1,0,0,0,0,0,0,0,2019-10-31 20:35:00,2019,10,31,20,20.583333


In [46]:
test_df_plus_holidays = pd.merge(test_df, holidays_df, 
                    on=["Year","Month","Day"], how='left')
test_df_plus_holidays["holiday"].fillna(0, inplace=True)    
test_df_plus_holidays

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,Amenity,Side,timestamp,Year,Month,Day,Hour,Time(h),holiday
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,0,0,2016-04-04 19:20:31,2016,4,4,19,19.341944,0.0
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,0,0,2020-10-28 11:51:00,2020,10,28,11,11.850000,0.0
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,0,0,2019-09-09 07:36:45,2019,9,9,7,7.612500,0.0
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,0,0,2019-08-06 15:46:25,2019,8,6,15,15.773611,0.0
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,0,0,2018-10-17 09:54:58,2018,10,17,9,9.916111,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,8003,37.812973,-122.362335,4.460,0,0,0,0,0,0,0,0,2020-06-26 22:32:22,2020,6,26,22,22.539444,0.0
1597,8004,37.761818,-122.405861,0.010,0,0,1,0,0,0,0,0,2016-12-03 07:16:30,2016,12,3,7,7.275000,0.0
1598,8005,37.732260,-122.431970,0.431,0,0,1,0,0,0,0,0,2017-02-20 06:32:44,2017,2,20,6,6.545556,1.0
1599,8006,37.786782,-122.390126,0.000,1,0,0,0,0,0,0,0,2019-10-31 20:35:00,2019,10,31,20,20.583333,0.0


In [47]:
test_df_plus_holidays["holiday"].value_counts()

0.0    1564
1.0      37
Name: holiday, dtype: int64

In [48]:
test_mergedDf = pd.merge(test_df_plus_holidays, df_pca, 
                    on=["Year","Month","Day", "Hour"], how='left')

test_mergedDf.isnull().sum()

ID              0
Lat             0
Lng             0
Distance(mi)    0
Crossing        0
Give_Way        0
Junction        0
No_Exit         0
Railway         0
Stop            0
Amenity         0
Side            0
timestamp       0
Year            0
Month           0
Day             0
Hour            0
Time(h)         0
holiday         0
pca_0           0
pca_1           0
pca_2           0
pca_3           0
dtype: int64

In [49]:
test_mergedDf.drop(columns=['Hour', 'timestamp'], inplace=True)
test_mergedDf

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,...,Side,Year,Month,Day,Time(h),holiday,pca_0,pca_1,pca_2,pca_3
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,...,0,2016,4,4,19.341944,0.0,9.473246,-11.690040,1.033256,3.552380
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,...,0,2020,10,28,11.850000,0.0,13.860088,12.436287,0.834647,-7.690887
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,...,0,2019,9,9,7.612500,0.0,-18.856600,-6.706959,-5.317522,-3.093242
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,...,0,2019,8,6,15.773611,0.0,16.400629,-9.800069,-12.924179,-1.201631
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,...,0,2018,10,17,9.916111,0.0,-9.269385,1.438045,0.751481,-5.315516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,8003,37.812973,-122.362335,4.460,0,0,0,0,0,0,...,0,2020,6,26,22.539444,0.0,-14.360472,10.104931,-9.981739,6.866996
1597,8004,37.761818,-122.405861,0.010,0,0,1,0,0,0,...,0,2016,12,3,7.275000,0.0,-2.655057,-12.270843,10.844791,-0.444604
1598,8005,37.732260,-122.431970,0.431,0,0,1,0,0,0,...,0,2017,2,20,6.545556,1.0,-15.455794,4.400080,-3.119496,-4.628654
1599,8006,37.786782,-122.390126,0.000,1,0,0,0,0,0,...,0,2019,10,31,20.583333,0.0,23.422204,15.634533,19.157629,0.838024


Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [50]:
X_test = test_mergedDf.drop(columns=['ID'])

# You should update/remove the next line once you change the features used for training
# X_test = X_test[['Lat', 'Lng', 'Distance(mi)']]

y_test_predicted = classifier.predict(X_test)

test_mergedDf['Severity'] = y_test_predicted

test_mergedDf.head()

,ID,Lat,Lng,Distance(mi),Crossing,Give_Way,Junction,No_Exit,Railway,Stop,...,Year,Month,Day,Time(h),holiday,pca_0,pca_1,pca_2,pca_3,Severity
0,6407,37.786060,-122.390900,0.039,0,0,1,0,0,0,...,2016,4,4,19.341944,0.0,9.473246,-11.690040,1.033256,3.552380,2
1,6408,37.769609,-122.415057,0.202,0,0,0,0,0,0,...,2020,10,28,11.850000,0.0,13.860088,12.436287,0.834647,-7.690887,2
2,6409,37.807495,-122.476021,0.000,0,0,0,0,0,0,...,2019,9,9,7.612500,0.0,-18.856600,-6.706959,-5.317522,-3.093242,2
3,6410,37.761818,-122.405869,0.000,0,0,1,0,0,0,...,2019,8,6,15.773611,0.0,16.400629,-9.800069,-12.924179,-1.201631,2
4,6411,37.732350,-122.414100,0.670,0,0,0,0,0,0,...,2018,10,17,9.916111,0.0,-9.269385,1.438045,0.751481,-5.315516,2


Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [51]:
test_mergedDf[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.